![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [2]:
import pandas as pd
import numpy as np
import calendar
# Start coding here...

In [3]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64
mortgage
--------------
mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64
previous_outcome
--------------
previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64
campaign_outcome
--------------
campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64


In [4]:
for col in ["month", "day"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

month
--------------
month
may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: count, dtype: int64
day
--------------
day
1     1450
5     1431
21    1404
3     1400
2     1400
17    1385
19    1376
25    1374
7     1373
22    1370
23    1366
6     1356
28    1339
9     1331
24    1327
16    1324
14    1324
27    1324
4     1323
20    1321
8     1311
13    1311
11    1306
15    1304
18    1303
26    1301
12    1298
29    1297
10    1282
30    1264
31     913
Name: count, dtype: int64


In [5]:
# Check the columns of the DataFrame to ensure 'cons_price_dx' is present
print(df.columns)

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')


In [6]:
print(df['previous_outcome'].value_counts())

previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64


In [7]:
economics = df[['client_id','cons_price_idx','euribor_three_months']]

In [8]:
economics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB


In [9]:
client = df[['client_id','age','job','marital','education','credit_default','mortgage']]

In [10]:
client['job'] = client['job'].str.replace(".","_")
client['education'] = client['education'].str.replace(".","_")
client['education'] = client['education'].replace('unknown', np.nan)

/var/folders/wv/gmqchw852_z_pqpsn82qqrnm0000gn/T/ipykernel_10289/1603334856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['job'] = client['job'].str.replace(".","_")
/var/folders/wv/gmqchw852_z_pqpsn82qqrnm0000gn/T/ipykernel_10289/1603334856.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['education'] = client['education'].str.replace(".","_")
/var/folders/wv/gmqchw852_z_pqpsn82qqrnm0000gn/T/ipykernel_10289/1603334856.py:3: SettingWithCopyWarning: 
A value is trying to be set on a co

In [11]:
client["credit_default"].value_counts()

credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64

In [12]:
client['credit_default'] = (client['credit_default'] == 'yes').astype(int).astype(bool)

/var/folders/wv/gmqchw852_z_pqpsn82qqrnm0000gn/T/ipykernel_10289/1764015162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['credit_default'] = (client['credit_default'] == 'yes').astype(int).astype(bool)


In [13]:
client['mortgage'] = (client['mortgage'] == "yes").astype(int).astype(bool)

/var/folders/wv/gmqchw852_z_pqpsn82qqrnm0000gn/T/ipykernel_10289/3070385109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['mortgage'] = (client['mortgage'] == "yes").astype(int).astype(bool)


In [14]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB


In [16]:
df['year'] = 2022

In [17]:
month_name_to_num = {month.lower(): index for index, month in enumerate(calendar.month_abbr) if month}
df['month']= df['month'].str.lower().map(month_name_to_num)

In [18]:
campaign = df[['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome']]

In [19]:
campaign['last_contact_date'] = pd.to_datetime(df[['year','month','day']]).dt.strftime('%Y-%m-%d')

/var/folders/wv/gmqchw852_z_pqpsn82qqrnm0000gn/T/ipykernel_10289/3019209473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['last_contact_date'] = pd.to_datetime(df[['year','month','day']]).dt.strftime('%Y-%m-%d')


In [21]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  object
 5   campaign_outcome            41188 non-null  object
 6   last_contact_date           41188 non-null  object
dtypes: int64(4), object(3)
memory usage: 2.2+ MB


In [22]:
campaign["campaign_outcome"].value_counts()

campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64

In [23]:
campaign["campaign_outcome"] = (campaign['campaign_outcome'] == "yes").astype(int).astype(bool)

/var/folders/wv/gmqchw852_z_pqpsn82qqrnm0000gn/T/ipykernel_10289/866722324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign["campaign_outcome"] = (campaign['campaign_outcome'] == "yes").astype(int).astype(bool)


In [24]:
campaign["previous_outcome"] = (campaign['previous_outcome'] == "success").astype(int).astype(bool)

/var/folders/wv/gmqchw852_z_pqpsn82qqrnm0000gn/T/ipykernel_10289/1647533417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign["previous_outcome"] = (campaign['previous_outcome'] == "success").astype(int).astype(bool)


In [25]:
campaign["previous_outcome"].value_counts()

previous_outcome
False    39815
True      1373
Name: count, dtype: int64

In [26]:
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"])

/var/folders/wv/gmqchw852_z_pqpsn82qqrnm0000gn/T/ipykernel_10289/2065111822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"])


In [27]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB


In [28]:
campaign.to_csv('campaign.csv',index=False)
client.to_csv('client.csv',index=False)
economics.to_csv('economics.csv',index=False)